# EPM Analysis
### Requires MossyEPM struct containing cell activity and behavior matrices for each mouse. Behavior is 30 fps, cell activity is 20 fps for drd73, drd77 and 10 fps for drd87, drd46

In [ ]:
def mc_epm_calc_beh_rates(behavior, cell, beh_type):
    """
    outputs rates and events per cell per behavior state specified
    calls on the 'define_beh_states' function to define behavior columns for
    different imaging sessions
    inputs behavior data, cell data, and specify what imaging session as
    'type'
    'type': (but see more behavior column detail in 'define_beh_states'
    'EPM'=elevated plus
    'EPMC'=EPM w center and open arms counted as one compartment
    'EZM'= elevated zero maze
    'OFT'= open field test
    'NOVEL'= novel object task
    """

[states, numb_beh, beh_length] = mc_epm_define_beh_states(behavior, beh_type);
"""
uses 'define_beh_states' function to define behavior columns to analyze
outputs the behavior indices in a cell array "states", how many behavior
states are extracted from that imaging session (numb_beh), and the length of time the
animal spent in that behavior state to use for rate calculations
(beh_length)

specificy if you want all the rates from 'define_beh_states' function, or
only rates from the 2 columns that are used in the bootstrap script
'all' gives all rates
'bootstrap' outputs the 2 rate columns used for bootstrap plus a
difference rate column 3

k = 1:size(cell, 2):
cellrates_behstates(k, numb_beh) = zeros
cellevents_behstates(k, numb_beh) = zeros
for b = 1:numb_beh:
    events_bycell = sum(cell(states{b}, :), 1)
    # loops through each behavioral state in cell array to sum events per cell in that state
    cellevents_behstates(k, b) = events_bycell
    # cells in rows, # of events in each behavior, each behavior in diff column

for b = 1:numb_beh
    cellrates_behstates(k, b) = cellevents_behstates(:, b)./beh_length(b);
    # calculates rate by dividing number of events by the length of that
    # behavioral state

return [cellrates_behstates, cellevents_behstates]
"""

def mc_epm_calc_beh_rates(behavior_dataframe):
    for column in behavior_dataframe:

In [ ]:
import pandas as pd

behavior_column_names = ['Trial time', 'Recording time', 'X center', 'Y center', 'Area', 'Areachange', 
                         'Elongation', 'Distance moved', 'Velocity', 'In zone(Arena / center-point)',
                         'In zone(Open1 / center-point)', 'In zone(Open2 / center-point)',
                         'In zone(Closed1 / center-point)', 'In zone(Closed2 / center-point)',
                         'In zone(OpenArms / center-point)', 'In zone(ClosedArms / center-point)', 'Result 1']

behavior_df = pd.read_csv('~/Behavior_Mouse_drd46.csv', header=None)
activity_df = pd.read_csv('~/activity_drd46.csv', header=None)

# Save only every nth row in order to downsample behavior Dataframes from 30fps -> 10fps
ROW_MULTIPLE = 3
behavior_df.drop(behavior_df.index[[i for i in range(0, len(behavior_df.index)) if i % ROW_MULTIPLE != 0]], inplace=True)

# Change column names to the behavior column names found in the MossyEPM, MATLAB struct
behavior_df.columns = behavior_column_names

# Fix indexing after downsample
behavior_df.reset_index(drop=True, inplace=True)

# Make the behavior Dataframe indexed by Recording time
# behavior_df.set_index('Recording time', inplace=True)

# Make the behavior Dataframe and the activity Dataframe have the same amount of rows 
if len(behavior_df.index) > len(activity_df.index):
    diff = len(behavior_df.index) - len(activity_df.index)
    behavior_df = behavior_df[:-diff]
elif len(behavior_df.index) < len(activity_df.index):
    diff = len(activity_df.index) - len(behavior_df.index)
    activity_df = activity_df[:-diff]

# Define running frames
VELOCITY_CUTOFF = 4;

# Adds column to the end of the behavior Dataframe and make each cell in that column a 0 
# if the corresponding velocity < VELOCITY_CUTOFF or a 1 if the corresponding velocity >= VELOCITY_CUTOFF
behavior_df['Running frames'] = [1 if velocity > VELOCITY_CUTOFF else 0 for velocity in behavior_df['Velocity'].tolist()]

# Calculate rates by arm, running, and time period

# Divide session into this many bins
TIME_BIN_FACTOR = 3; 

"""
activity = EPM_activity.(mice{i});
behavior = EPM_behavior_ds.(mice{i});

# Rates for entire session
[AUCrates.(mice{i}), ~] = mc_epm_calc_beh_rates(behavior, activity, 'EPM');
AUCrates.(mice{i}) = 10 * AUCrates.(mice{i});

# AUCrates_binned = NaN;

bin_length = floor(size(activity, 1) / time_bin_factor);

for j = in range(0, TIME_BIN_FACTOR):
    start_frame = (j - 1) * bin_length + 1;
    end_frame = j * bin_length;
    [AUCrates_binned.(mice{i}){j}, AUCtotal_binned.(mice{i}){j}] = mc_epm_calc_beh_rates(behavior(start_frame:end_frame, :), activity(start_frame:end_frame, :), 'EPM');  

    # Multiply by frame rate to get rate in seconds
    AUCrates_binned.(mice{i}){j} = 10 * AUCrates_binned.(mice{i}){j}; 
    
    # Divide by bin_length to get total average total rate per bin
    # AUCtotal_binned.(mice{i}){j} = 10*AUCtotal_binned.(mice{i}){j} / bin_length;
    
    AUCtotal_binned_rate.(mice{i}){j} = 10 * sum(AUCtotal_binned.(mice{i}){j}(:, 1:3), 2) / bin_length; 

# AUCtotal_binned_rate.(mice{i}) = cell2mat(AUCtotal_binned_rate.(mice{i}));

# Create array with all cells by behavior over the whole session 
for i = 1:numel(mice)
    if i == 1:
        AUC_all_cells = AUCrates.(mice{i});
    else:
        AUC_all_cells = [AUC_all_cells; AUCrates.(mice{i})]

# Create array with all neurons for each time bin, irrespective of behavior
for i = 1:numel(mice):
    if i == 1:
        AUC_bin_all_cells = AUCtotal_binned_rate.(mice{i})
    else:
        AUC_bin_all_cells = [AUC_bin_all_cells; AUCtotal_binned_rate.(mice{i})]

# Calculate time bin averages and SEM per animal
for i = 1:numel(mice):
    AUC_all_cells_binned = AUCrates.rates_binned.(mice{i});
    AUC_bin_means(i,:) = mean(AUCtotal_binned_rate.(mice{i}));
    AUC_bin_SE(i,:) = std(AUCtotal_binned_rate.(mice{i}))/sqrt(size(AUCrates.(mice{i}),1));

# Store 10 col. averages per time bin (Nbins x 10)
for i = 1:numel(mice):
    for j in range(0, TIME_BIN_FACTOR):
        sum_closed(j) = sum_closed(j) + sum(AUCrates_binned.(mice{i}){j}(1), 2);
        sum_open(j) = sum_open(j) + sum(AUCrates_binned.(mice{i}){j}(2), 2);    
"""

# *** TEST ***
behavior_df
# activity_df